In [36]:
import pandas as pd
import numpy as np

In [27]:
data = pd.read_csv('movie_prediction_data.csv')

In [28]:
## Taking out the first row of data for testing
test_row = data.head(1)

In [29]:
## Resetting the dataframe excluding the top row
data = data.iloc[1:]
data = data.reset_index(drop = True)

In [30]:
data.head(5)

,title,production,release_year,genre1,genre2,genre3,director1,director2,actor1,actor2,actor3,budget,production_country
0,Hereditary,PalmStar Media,2018,Drama,Horror,Mystery,Ari Aster,NaN,Alex Wolff,Gabriel Byrne,Toni Collette,10.0,US
1,Unfriended: Dark Web,Bazelevs Production,2018,Horror,NaN,NaN,Stephen Susco,NaN,Rebecca Rittenhouse,Betty Gabriel,Colin Woodell,1.0,RU
2,Split,Blinding Edge Pictures,2016,Horror,Thriller,NaN,M. Night Shyamalan,NaN,James McAvoy,Anya Taylor-Joy,Betty Buckley,9.0,US
3,Truth or Dare,Blumhouse Productions,2018,Horror,Thriller,NaN,Jeff Wadlow,NaN,Lucy Hale,Tyler Posey,Violett Beane,3.5,US
4,The Row,Emmett/Furla/Oasis Films (EFO Films),2018,Thriller,NaN,NaN,Matty Beckerman,NaN,Colin Egglesfield,Mia Rose Frampton,Lala Kent,4.0,US


# Method 1: Using goodall4 and categorical match with taking budget into consideration

In [42]:
def goodall4(data):
        score = np.zeros((data.shape[0], data.shape[0]))
        p_2_k_d = [dict(zip(np.unique(data[:, x], return_counts=True)[0], 1.*np.unique(data[:, x], return_counts=True)[1]*(np.unique(data[:, x], return_counts=True)[1]-1)/data.shape[0]/(data.shape[0]-1))) for x in range(data.shape[1])]
        for i, X in enumerate(data):
            tmp_bool = data==X
            tmp_x = np.zeros(data.shape)
            for k in range(data.shape[1]):
                tmp_x[:, k][tmp_bool[:, k] == True] = 1. * p_2_k_d[k].get(X[k], 0) / data.shape[1]
            score[i] = np.sum(tmp_x, axis=1)
        return score[0][0]

In [33]:
## considering only title, production and production_country for goodall4 measure
totaldata = data[['title','production','production_country']].values

In [44]:
testrow_cat = test_row[['title', 'production', 'production_country']].values.tolist()[0]
testrow_gen = test_row[['genre1', 'genre2', 'genre3']].values.tolist()[0]
testrow_dir = test_row[['director1', 'director2']].values.tolist()[0]
testrow_actr = test_row[['actor1', 'actor2', 'actor3']].values.tolist()[0]
similarity_dict = {}
alldata = np.vstack([totaldata, test_row[['title', 'production', 'production_country']].values[0]])
#p_2_k_d = [dict(zip(np.unique(alldata[:, x], return_counts=True)[0], 1.*np.unique(alldata[:, x], return_counts=True)[1]*(np.unique(alldata[:, x], return_counts=True)[1]-1)/alldata.shape[0]/(alldata.shape[0]-1))) for x in range(alldata.shape[1])]
for index,row  in data.iterrows():
    cat = (row[['title', 'production', 'production_country']].values.tolist())
    gen = row[['genre1', 'genre2', 'genre3']].astype(str).values.tolist()
    dire = row[['director1', 'director2']].astype(str).values.tolist()
    actr = row[['actor1', 'actor2', 'actor3']].astype(str).values.tolist()
    temp_data = np.array([testrow_cat, cat])
    goodallsim = goodall4(temp_data)
    gensim = len(set(testrow_gen)&set(gen)) / float(len(set(testrow_gen) | set(gen)))
    dirsim = len(set(testrow_dir)&set(dire)) / float(len(set(testrow_dir) | set(dire)))
    actrsim = len(set(testrow_actr)&set(actr)) / float(len(set(testrow_actr) | set(actr)))
    budgetsim = 1-abs((float(row.budget) - float(test_row.budget))/20)
    #print(budgetsim)
    score = (0.2*goodallsim)+(0.25*gensim)+(0.15*dirsim)+(0.2*actrsim)+(0.2*budgetsim)
    similarity_dict[index] = score
    
#print(simdict)

In [47]:
## get the id of row with max similarity score
max(similarity_dict, key=similarity_dict.get)

100

In [48]:
data.iloc[100]

title                  Batman: The Killing Joke
production              Warner Bros. Animation 
release_year                               2016
genre1                               Animation 
genre2                                  Action 
genre3                                    Crime
director1                           Sam Liu (I)
director2                                   NaN
actor1                             Kevin Conroy
actor2                              Mark Hamill
actor3                              Tara Strong
budget                                      3.5
production_country                           US
Name: 100, dtype: object

# Method 2: Using CountVectorizer and Cosine Similarity(budget not considered)

In [49]:
df = pd.read_csv('movie_prediction_data.csv')

In [50]:
df.shape

(8070, 13)

In [51]:
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet

In [ ]:
## Merging all genres, directors and actors together respectively

In [52]:
df['genres']=df['genre1'].astype(str)+','+df['genre2'].astype(str)+','+df['genre3'].astype(str)

In [53]:
df['directors']=df['director1'].astype(str)+','+df['director2'].astype(str)

In [54]:
df['actors']=df['actor1'].astype(str)+','+df['actor2'].astype(str)+','+df['actor3'].astype(str)

In [ ]:
## Dropping the individual columns after merging

In [55]:
df.drop(['genre1','genre2','genre3','director1','director2','actor1','actor2','actor3'], axis=1, inplace=True)

In [ ]:
## Converting the actor names to lowercase and also stripping space between firstname and lastname

In [56]:
df['actors'] = df['actors'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [57]:
df['genres'] = df['genres'].apply(lambda x: str.lower(x.replace(" ", "")))

In [63]:
df['directors'] = df['directors'].apply(lambda x: str.lower(x.replace(" ", "")))

In [58]:
df['production'] = df['production'].astype(str).apply(lambda x: str.lower(x.replace(" ", "")))

In [59]:
df['production_country'] = df['production_country'].astype(str).apply(lambda x: str.lower(x.replace(" ", "")))

In [ ]:
## Creating tokens of all columns 

In [64]:
df['soup'] = df['actors'] +" "+ df['directors'] +" "+ df['genres']+" "+df['production']+" "+df['production_country']

In [65]:
df['soup'] = df['soup'].apply(lambda x: x.replace(","," "))

In [66]:
df.soup[0]

'kristenbell nicolascage tarastrong aaronhorvath peterridamichail animation action comedy warnerbros.animation us'

In [67]:
## Make a count matrix of all tokens

In [68]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [69]:
count_matrix = count.fit_transform(df['soup'])

In [70]:
## Calculate cosine similarity of count matrix

In [71]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [72]:
df = df.reset_index()
titles = df['title']
indices = pd.Series(df.index, index=df['title'])

In [73]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [74]:
## Testing the results

In [75]:
## Get top 5 recommendations for given movie
get_recommendations('Hereditary').head(5)

599              Armenian Haunting
673                      SheChotic
2947                       #Horror
3000    The Curse of Downers Grove
3741                        Everto
Name: title, dtype: object

In [76]:
df.iloc[599]

index                                                               599
title                                                 Armenian Haunting
production                                           reelnightmarefilms
release_year                                                       2018
budget                                                             0.03
production_country                                                   us
genres                                             drama,horror,mystery
directors                                          artarutyunyan(i),nan
actors                vanehassadourian,jaydersahagian,kylepatrickdar...
soup                  vanehassadourian jaydersahagian kylepatrickdar...
Name: 599, dtype: object

# Method 3: Same as above by taking budget into consideration (Recommended)

In [81]:
## Takes the top 25 similar movies and calculates the budget difference and returns the top 10 movies 
## having minimum budget difference. The similarity will give better results when we use plot keywords and rating too.
def recommender_by_budget(title):
    idx = indices[title]
    #print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    recommended_movies = df.iloc[movie_indices][['title','budget']]
    movie = df.iloc[idx][['title','budget']]
    movie_budget = float(movie.budget)
    recommended_movies['budget_sim'] = recommended_movies['budget'].apply(lambda x: 1-abs((x-movie_budget)/20))
    recommended_movies = recommended_movies.sort_values('budget_sim', ascending=False).head(10)
    return recommended_movies.title

In [85]:
recommender_by_budget('Teen Titans Go! To the Movies')

104                          Woody Woodpecker
211                         Norm of the North
252             Bad Kids of Crestview Academy
7589                            November 10th
3044       Alpha and Omega 5: Family Vacation
2961          Dragon Ball Z: Resurrection 'F'
1634     Three heroes and the King of the Sea
169                             Don't Kill It
222     Batman: Return of the Caped Crusaders
101                  Batman: The Killing Joke
Name: title, dtype: object

In [86]:
df.iloc[104]

index                                                               104
title                                                  Woody Woodpecker
production                                   universal1440entertainment
release_year                                                       2017
budget                                                               10
production_country                                                   us
genres                                          animation,comedy,family
directors                                                  alexzamm,nan
actors                            timothyomundson,thailaayala,ericbauza
soup                  timothyomundson thailaayala ericbauza alexzamm...
Name: 104, dtype: object